# **EKF Simulation**
In questo notebook implementeremo un EKF avanzato con tecniche di correzione del coning, sculling delayed state.

In [1]:
import numpy as np
import math
import matplotlib

## Utilities

### LPF 1 ordine esponenziale

$$\alpha = exp(-\frac{2\pi f_c}{f_s})$$

$$y_k = \alpha y_{k-1} + (1 - \alpha)x_k$$

In [ ]:
def calculate_alpha(fc, fs):  #fc = cutoff frequency | fs = sample frequency
    return math.exp(-((2*math.pi*fc) / (fs)))

def lpfFirstOrderExp(x, y_prev, alpha):    #x = new measure
    y = alpha * y_prev + (1 - alpha) * x
    lpfFirstOrderExp.y_prev = y

    return y

### Coning Compensation

$$\Delta\theta _{k} = \omega dt$$
$$\Delta\theta _{corr} = \Delta\theta _{k-1} + \Delta\theta _{k} + \frac{1}{12}(\Delta\theta _{k-1} \times \Delta\theta _{k} )$$

In [3]:
def coning_compensation(omega, omega_prev, dt):
    dtheta = omega * dt
    dtheta_prev = omega_prev * dt

    return dtheta_prev + dtheta + 1/12*(np.cross(dtheta_prev, dtheta))

### Sculling Compensation


$$\Delta v_{k} = a dt$$
$$\Delta v_{corr} = \Delta v_{k-1} + \Delta v_{k} + \frac{1}{2}(\Delta\theta _{k-1} \times \Delta v_{k} ) + \frac{1}{12}(\Delta\theta _{k-1} \times \Delta v_{k-1} )$$

In [4]:
def sculling_compensation(acc, acc_prev, omega_prev, dt):
    dv = acc * dt
    dv_prev = acc_prev * dt
    dtheta_prev = omega_prev * dt

    return dv_prev + dv + 1/2*np.cross(dtheta_prev, dv) + 1/12*np.cross(dtheta_prev, dv_prev)